In [ ]:
!nvidia-smi

Tue Aug 10 09:29:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
# Mount the Google Drive
drive.mount('/content/drive', force_remount=True)
import os
# Now, cd to a folder from my Google Drive
os.chdir("/content/drive/My Drive/Chris/3D-ML/Open3D-ML_master")

In [ ]:
!pip3 install open3d # 0.13.0
!pip3 install tensorflow~=2.4.1 # -r requirements-tensorflow.txt
!pip3 install laspy[lazrs] # 2.0.2 

## Amsterdam dataloader
We will make this in a Python script.

NOTE: Comment line 60 (raise keyerror) in the registry.py in dist-packages/tensorflow. You can open this in Colab. Restart the runtime after. 

In [ ]:
import os
import ml3d as _ml3d
import ml3d.tf as ml3d

cfg_file = "ml3d/configs/randlanet_amsterdam3d.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.RandLANet(**cfg.model)
cfg.dataset['dataset_path'] = "/content/drive/My Drive/Datasets/street_lights/LiDAR/Weesp_auto_labeling/Cyclomedia_pc_verified"
dataset = _ml3d.datasets.Amsterdam3D(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model=model, dataset=dataset, max_epoch=200, batch_size=1, device='gpu')

ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)

In [ ]:
pipeline.cfg_tb = {
    "readme": "readme",
    "cmd_line": "cmd_line",
    "dataset": "Amsterdam3D",
    "model": "RandLaNet",
    "pipeline": "Default Pipeline",
}

In [ ]:
pipeline.run_train()

## Evaluate

In [ ]:
!pip3 install addict

In [ ]:
import os
import ml3d as _ml3d
import ml3d.tf as ml3d

cfg_file = "ml3d/configs/randlanet_amsterdam3d.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.RandLANet(**cfg.model)
cfg.dataset['dataset_path'] = "/content/drive/My Drive/Datasets/street_lights/LiDAR/Weesp_auto_labeling/Cyclomedia_pc_verified"
dataset = _ml3d.datasets.Amsterdam3D(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

# pretrained model
ckpt_path = "/content/drive/My Drive/Chris/Open3D-ML/Open3D-ML_master/logs/RandLANet_Amsterdam3D_tf/checkpoint/ckpt-11"

# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

test_split = dataset.get_split("val")
data = test_split.get_data(0)

# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)

# evaluate performance on the test set; this will write logs to './logs'.
pipeline.run_test()

## Visualize

In [ ]:
# based on https://github.com/intel-isl/Open3D-ML/blob/master/examples/vis_pred.py
import numpy as np
import os
import ml3d as _ml3d
import ml3d.tf as ml3d
import sys
from os.path import exists, join, isfile, dirname, abspath, split
import laspy

def get_custom_data(pc_names, path):
    pc_data = []
    for i, name in enumerate(pc_names):
        pc_path = join(path, 'points', name)

        data = laspy.read(pc_path)

        points = np.vstack((data.x, data.y, data.z)).T.astype(np.float32)

        feat = np.vstack((data.red, data.green, data.blue)).T.astype(np.float32)
        
        label = np.zeros((points.shape[0],), dtype=np.int32)
        
        data = {
            'point': points,
            'feat': feat,
            'label': label,
        }
        pc_data.append(data)

    return pc_data


def pred_custom_data(pc_names, pcs, pipeline_r):
    vis_points = []
    for i, data in enumerate(pcs):
        name = pc_names[i]

        results_r = pipeline_r.run_inference(data)
        pred_label_r = (results_r['predict_labels'] + 1).astype(np.int32)

        # WARNING, THIS IS A HACK
        # Fill "unlabeled" value because predictions have no 0 values.
        pred_label_r[0] = 0

        label = data['label']
        pts = data['point']

        vis_d = {
            "name": name + "_randlanet",
            "points": pts,
            "labels": pred_label_r,
        }
        vis_points.append(vis_d)

    return vis_points

pc_names = ["processed_2633_9595.laz"]

cfg_file = "openje/ml3d/configs/randlanet_amsterdam3d.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

amsterdam_labels = _ml3d.datasets.Amsterdam3D.get_label_to_names()
v = _ml3d.vis.Visualizer()
lut = _ml3d.vis.LabelLUT()
for val in sorted(amsterdam_labels.keys()):
    lut.add_label(amsterdam_labels[val], val)
data_path = "demo_data_amsterdam"
cfg.dataset["dataset_path"] = data_path

v.set_lut("labels", lut)
v.set_lut("pred", lut)

ckpt_path = "checkpoint/ckpt-1"
model = ml3d.models.RandLANet(**cfg.model)
pipeline_r = ml3d.pipelines.SemanticSegmentation(model, **cfg.pipeline)
pipeline_r.load_ckpt(ckpt_path)

pcs = get_custom_data(pc_names, data_path)
pcs_with_pred = pred_custom_data(pc_names, pcs, pipeline_r)

v.visualize(pcs_with_pred)